In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\USER\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:

import numpy as np 
import pandas as pd
import json

# Load Dataset

In [5]:
meta = pd.read_csv('trip.csv')

meta.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"['Animation'}, {'Comedy'}, {'Family'}]",http://toystory.disney.com/toy-story,1,tt0114709,en,DOKYO,"Led by Woody, Andy's toys live happily in his ...",...,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033,81,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,DOKYO
1,False,NaN,65000000,"['FANTASY'}, {'Comedy'}, {'Family'}]",NaN,2,tt0113497,en,KOREA,When siblings Judy and Peter discover an encha...,...,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249,104,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"['Animation'}, {'ROMANCE'}, {'Family'}]",NaN,3,tt0113228,en,MADRID,A family wedding reignites the ancient feud be...,...,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0,101,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men
3,False,NaN,16000000,"['Animation'}, {'Comedy'}, {'Family'}]",NaN,4,tt0114885,en,PARI,"Cheated on, mistreated and stepped on, the wom...",...,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156,127,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"['DRAMA'}, {'Comedy'}, {'Family'}]",NaN,5,tt0113041,en,BUSAN,Just when George Banks has recovered from his ...,...,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911,106,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II


In [6]:
meta = meta[['id', 'original_title', 'original_language', 'genres']]
meta = meta.rename(columns={'id':'tripId'})
meta = meta[meta['original_language'] == 'en']
meta.head()

,tripId,original_title,original_language,genres
0,1,DOKYO,en,"['Animation'}, {'Comedy'}, {'Family'}]"
1,2,KOREA,en,"['FANTASY'}, {'Comedy'}, {'Family'}]"
2,3,MADRID,en,"['Animation'}, {'ROMANCE'}, {'Family'}]"
3,4,PARI,en,"['Animation'}, {'Comedy'}, {'Family'}]"
4,5,BUSAN,en,"['DRAMA'}, {'Comedy'}, {'Family'}]"


In [9]:
ratings = pd.read_csv('triprating.csv')
ratings = ratings[['userId', 'tripId', 'rating']]
ratings.head()

,userId,tripId,rating
0,0,1,5.0
1,78,2,3.0
2,2,3,4.0
3,3,4,5.0
4,4,5,5.0


In [10]:
ratings.describe()

,userId,tripId,rating
count,14.000000,14.000000,14.000000
mean,10.642857,3.642857,3.178571
std,22.578435,1.598420,1.771648
min,0.000000,1.000000,1.000000
25%,0.000000,2.250000,1.250000
50%,3.000000,3.500000,3.500000
75%,4.750000,5.000000,5.000000
max,78.000000,6.000000,5.000000


# Refine Dataset

In [11]:
meta.tripId = pd.to_numeric(meta.tripId, errors='coerce')
ratings.tripId = pd.to_numeric(ratings.tripId, errors='coerce')

In [14]:
def parse_genres(genres_str):
    genres = json.loads(genres_str.replace('\'', '"'))
    
    genres_list = []
    for g in genres:
        genres_list.append(g['name'])

    return genres_list

meta['genres'] = meta['genres'].apply(parse_genres)

meta.head()

JSONDecodeError: Expecting ',' delimiter: line 1 column 13 (char 12)

# Merge Meta and Ratings

In [16]:
data = pd.merge(ratings, meta, on='tripId', how='inner')

data.head()

,userId,tripId,rating,original_title,original_language,genres
0,0,1,5.0,DOKYO,en,"['Animation'}, {'Comedy'}, {'Family'}]"
1,78,2,3.0,KOREA,en,"['FANTASY'}, {'Comedy'}, {'Family'}]"
2,4,2,2.0,KOREA,en,"['FANTASY'}, {'Comedy'}, {'Family'}]"
3,0,2,1.0,KOREA,en,"['FANTASY'}, {'Comedy'}, {'Family'}]"
4,2,3,4.0,MADRID,en,"['Animation'}, {'ROMANCE'}, {'Family'}]"


# Pivot Table

In [17]:
matrix = data.pivot_table(index='userId', columns='original_title', values='rating')

matrix.head(20)

original_title,BUSAN,DOKYO,KOREA,MADRID,PARI
userId,,,,,
0,4.5,5.0,1.0,1.0,2.0
2,NaN,NaN,NaN,4.0,NaN
3,NaN,NaN,NaN,5.0,5.0
4,5.0,NaN,2.0,NaN,NaN
5,1.0,NaN,NaN,NaN,NaN
45,5.0,NaN,NaN,NaN,NaN
78,NaN,NaN,3.0,NaN,NaN


# Pearson Correlation

https://namu.wiki/w/%EC%83%81%EA%B4%80%20%EA%B3%84%EC%88%98?from=%ED%94%BC%EC%96%B4%EC%8A%A8%20%EC%83%81%EA%B4%80%20%EA%B3%84%EC%88%98#s-2

In [18]:
GENRE_WEIGHT = 0.1

def pearsonR(s1, s2):
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))

def recommend(input_movie, matrix, n, similar_genre=True):
    input_genres = meta[meta['original_title'] == input_movie]['genres'].iloc(0)[0]

    result = []
    for title in matrix.columns:
        if title == input_movie:
            continue

        # rating comparison
        cor = pearsonR(matrix[input_movie], matrix[title])
        
        # genre comparison
        if similar_genre and len(input_genres) > 0:
            temp_genres = meta[meta['original_title'] == title]['genres'].iloc(0)[0]

            same_count = np.sum(np.isin(input_genres, temp_genres))
            cor += (GENRE_WEIGHT * same_count)
        
        if np.isnan(cor):
            continue
        else:
            result.append((title, '{:.2f}'.format(cor), temp_genres))
            
    result.sort(key=lambda r: r[1], reverse=True)

    return result[:n]

# Prediction

In [24]:
recommend_result = recommend('KOREA', matrix, 10, similar_genre=True)

pd.DataFrame(recommend_result, columns = ['Title', 'Correlation', 'Genre'])

C:\Users\USER\AppData\Local\Temp\ipykernel_39780\2882930674.py:6: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))


,Title,Correlation,Genre
0,MADRID,0.56,"['Animation'}, {'ROMANCE'}, {'Family'}]"
1,PARI,0.50,"['Animation'}, {'Comedy'}, {'Family'}]"
2,BUSAN,-0.13,"['DRAMA'}, {'Comedy'}, {'Family'}]"
